In [1]:
import petl as etl
from collections import OrderedDict

In [2]:
src = [['SampleId', 'TOC', 'Method'],
       ['s1', 0.5, 'leco'],
       ['s2', 0.3, 'pyrol'],
       ['s3', 0.1, None]]
t1 = etl.wrap(src)
t1

SampleId,TOC,Method
s1,0.5,leco
s2,0.3,pyrol
s3,0.1,None


Could use DSL to output mapping dicts that we could use for col expansion?

In [3]:
# manually produced example of the code we would need to generate to map...
def row_mapper(row):
    def expand(src_col, ref_col, rc_val):
        return row[src_col] if row[ref_col] == rc_val else None
    
    return [
        row[0], # ids unchanged
        # could build a collection of the mapping below and unpack with *args?, need one of below per exp col / ref val comb
        expand('TOC', 'Method', 'leco'),
        expand('TOC', 'Method', 'pyrol'),
        expand('TOC', 'Method', None)
           ]
toc_grps = [f'toc.{m}' if m else 'toc.un' for m in t1['Method']]  # one header for each toc mapping
t2 = etl.rowmap(t1, row_mapper, header=['SampleId'] + toc_grps)
t2

SampleId,toc.leco,toc.pyrol,toc.un
s1,0.5,None,None
s2,None,0.3,None
s3,None,None,0.1


In [9]:
# alt - could use unpack: https://petl.readthedocs.io/en/stable/transform.html#unpacking-compound-values
#       i.e. convert to a seq of values in place then unpack this to multi cols?
#       or unpackdict: https://petl.readthedocs.io/en/stable/transform.html#petl.transform.unpacks.unpackdict
src_col, ref_col = 'TOC', 'Method'
method_to_a_grp = {m: m if m else 'un' for m in t1[ref_col]}

def build_mapping_dict(v, row):
    return {f"{src_col}.{v}": row[src_col] if k == row[ref_col] else "" 
            for k, v in method_to_a_grp.items()}

t1_dict = etl.convert(t1, 'TOC', build_mapping_dict)    
t1_dict

SampleId,TOC,Method
s1,None,leco
s2,None,pyrol
s3,None,None


In [5]:
# next steps - bring in DSL core types & try to hook up to code generator 
#              that produces code like the above?
# consider property model - may need to mock initially?!?

In [6]:
#mappings = OrderedDict()
#mappings['SampleId'] = 'SampleId'
#method_to_a_grp = {m: m if m else 'un' for m in t1['Method']} # real would use PM

#def get_expander(src_col, ref_col, row):
#    rc_val = method_to_a_grp[row[ref_col]]
#    result = row[src_col] if row[ref_col] == rc_val else None
#    print(f"rc_val: {rc_val}, row: {row} | result => {result}")
#    return result

## cretae a column (expand toc) for each analyis group 
#for grp in method_to_a_grp.values():
#    converter = lambda row: get_expander('TOC', 'Method', row)
#    mappings[f'toc.{grp}'] = converter

## not working yet - need to only populate value when the analysis 
##                   is a match for the method mapping